In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
path_to_data = "raw_data"

In [ ]:
def parse_inter_name(name):
    for direction_name in ["东口", "西口", "南口", "北口", "东向西", "西向东", "南向北", "北向南"]:
        if direction_name in name:
            index_direction = name.index(direction_name)
            return name[:index_direction]
    
    return "非地面路口" 
    
def parse_entrance_name(name):
    for direction_name in ["东口", "西口", "南口", "北口", "东向西", "西向东", "南向北", "北向南"]:
        if direction_name in name:
            return direction_name
        
    return "非地面路口" 

In [ ]:
# list_df = []
# for file in os.listdir(path_to_data):
#     if file.startswith("list_dev_id_part_"):
#         list_df.append(pd.read_csv(os.path.join(path_to_data, file))[["dev_id", "dev_name"]])
# df = pd.concat(list_df)
# df.drop_duplicates(inplace=True)
df = pd.read_csv(os.path.join(path_to_data, "name.txt"), delimiter="\t")

In [ ]:
df["inter_name"] = df["dev_name"].apply(lambda r: parse_inter_name(r))
df["entrance"] = df["dev_name"].apply(lambda r: parse_entrance_name(r))

In [ ]:
df = df[df["inter_name"] != "非地面路口"]
df.reset_index(inplace=True)
df.to_csv(os.path.join(path_to_data, "list_dev_id_total.csv"))

In [ ]:
# get directions count
list_candidate_entrance = ["东口", "西口", "南口", "北口", "东向西", "西向东", "南向北", "北向南"]
list_inter_entrance_mapping = []
list_inter_names = np.unique(df["inter_name"].values)
for inter_name in list_inter_names:
    list_entrance = df["entrance"][df["inter_name"] == inter_name].values
    array_inter = [inter_name]
    for i, d in enumerate(list_candidate_entrance):
        if d in list_entrance:
            array_inter.append(1)
        else:
            array_inter.append(0)
    list_inter_entrance_mapping.append(array_inter)
df_inter_entrance_mapping = pd.DataFrame(list_inter_entrance_mapping, columns=["inter_name"]+list_candidate_entrance)
    

In [ ]:
df_inter_entrance_mapping.to_csv(os.path.join(path_to_data, "list_inter_entrance_mapping.csv"))

In [ ]:
list_road_to_include = ["湖墅"]
list_row_to_include = []
for road in list_road_to_include:
    for i in range(len(df["inter_name"])):
        if road in df["inter_name"][i]:
            list_row_to_include.append(i)
list_road_to_include = list(set(list_road_to_include))

In [ ]:
df.loc[list_row_to_include].to_csv(os.path.join(path_to_data, "list_dev_id_select.csv"))
print(df.loc[list_row_to_include]["dev_id"].values.tolist())